# Introduction to Dask

In [ ]:
!pip install --upgrade dask distributed bokeh adlfs fsspec fastparquet pyarrow python-snappy lz4 gitpython

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# setup data path
data_path = prefix.joinpath("data", "raw", "iris", "iris.csv")

In [ ]:
from dask.distributed import Client

c = Client()
c

## Pandas and Dask on local data

In [ ]:
import pandas as pd
import dask.dataframe as dd

In [ ]:
%%time
df = pd.read_csv(data_path)

In [ ]:
%%time
ddf = dd.read_csv(data_path)

In [ ]:
%%time
df.describe()

In [ ]:
%%time
ddf.describe().compute()

## Read data from Azure

In [ ]:
ds = ws.get_default_datastore()
ds.upload_files([str(data_path)], target_path="datasets/iris", overwrite=True)

In [ ]:
container_name = ds.container_name

storage_options = {
    "account_name": ds.account_name,
    "account_key": ds.account_key,
}

In [ ]:
%%time
ddf = dd.read_csv(
    f"az://{container_name}/datasets/iris/iris.csv",
    storage_options=storage_options,
)

In [ ]:
%%time
ddf.describe().compute()

## Convert to Pandas

In [ ]:
%%time
df = ddf.compute()

In [ ]:
%%time
df.describe()

## Bigger data

In [ ]:
container_name = "isdweatherdatacontainer"

storage_options = {"account_name": "azureopendatastorage"}

In [ ]:
from adlfs import AzureBlobFileSystem

fs = AzureBlobFileSystem(**storage_options)
fs

In [ ]:
%%time
fs.ls(f"{container_name}/ISDWeather/year=2020")

In [ ]:
files = fs.glob(f"{container_name}/ISDWeather/year=2020/month=2/*.parquet")
files = [f"az://{file}" for file in files]
files[-5:]

In [ ]:
%%time
ddf = dd.read_parquet(files, storage_options=storage_options, chunksize="20MB")
ddf

In [ ]:
ddf = ddf.persist()

In [ ]:
%%time
len(ddf)

In [ ]:
%%time
len(ddf)

In [ ]:
%%time
ddf.describe().compute()

In [ ]:
%%time
gbs = round(ddf.memory_usage(index=True, deep=True).sum().compute() / 1e9, 2)
print(f"ddf is {gbs} GBs")